In [27]:
import requests
import pandas as pd
import numpy as np

In [28]:
team_id = 2723076
# the gw you want to predict minus 1
gameweek = 20

In [29]:
# link structure: https://fantasy.premierleague.com/api/entry/2723076/event/19/picks/
api_link = "https://fantasy.premierleague.com/api/entry/"

In [30]:
response = requests.get(f"{api_link}/{team_id}/event/{gameweek}/picks/")
api_data = response.json()
players = api_data.get("picks", [])
# players is a list of dictionaries, w each dict representing one player in the team
players

[{'element': 352,
  'position': 1,
  'multiplier': 1,
  'is_captain': False,
  'is_vice_captain': False},
 {'element': 506,
  'position': 2,
  'multiplier': 1,
  'is_captain': False,
  'is_vice_captain': False},
 {'element': 265,
  'position': 3,
  'multiplier': 1,
  'is_captain': False,
  'is_vice_captain': False},
 {'element': 31,
  'position': 4,
  'multiplier': 1,
  'is_captain': False,
  'is_vice_captain': False},
 {'element': 526,
  'position': 5,
  'multiplier': 1,
  'is_captain': False,
  'is_vice_captain': False},
 {'element': 516,
  'position': 6,
  'multiplier': 2,
  'is_captain': True,
  'is_vice_captain': False},
 {'element': 344,
  'position': 7,
  'multiplier': 1,
  'is_captain': False,
  'is_vice_captain': False},
 {'element': 362,
  'position': 8,
  'multiplier': 1,
  'is_captain': False,
  'is_vice_captain': False},
 {'element': 34,
  'position': 9,
  'multiplier': 1,
  'is_captain': False,
  'is_vice_captain': False},
 {'element': 590,
  'position': 10,
  'multiplier

In [31]:
team_IDs = [player["element"] for player in players]
team_IDs

[352, 506, 265, 31, 526, 516, 344, 362, 34, 590, 85, 113, 430, 355, 511]

In [32]:
# reading in the list of player IDs to convert IDs to names
player_IDs = pd.read_csv("player_IDs.csv")
player_IDs

,first_name,second_name,id
0,Folarin,Balogun,1
1,Cédric,Alves Soares,2
2,Mohamed,Elneny,3
3,Fábio,Ferreira Vieira,4
4,Gabriel,dos Santos Magalhães,5
...,...,...,...
753,Nathan,Fraser,704
754,Jean-Ricner,Bellegarde,715
755,Matthew,Whittingham,749
756,Tawanda,Chirewa,750


In [33]:
# format: https://fantasy.premierleague.com/api/element-summary/{player_id}/
player_api_link = "https://fantasy.premierleague.com/api/element-summary/"

In [34]:
to_add_df = pd.DataFrame()
for player_id in team_IDs: 

    player_row = player_IDs[player_IDs["id"] == player_id]
    player_name = f"{player_row['first_name'].iloc[0]} {player_row['second_name'].iloc[0]}"
    response = requests.get(f"{player_api_link}{player_id}/")
    api_data = response.json()
    fixture_stats = pd.DataFrame(api_data.get("history", []))
    # add another row for upcoming game we are predicting pts for
    fixture_stats.loc[len(fixture_stats)] = 0
    future_fixtures = pd.DataFrame(api_data.get("fixtures", []))
    # get the row for this GW
    this_gw = future_fixtures[future_fixtures["event_name"] == f"Gameweek {gameweek + 1}"]
    fixture_stats.loc[fixture_stats.index[-1], "kickoff_time"] = this_gw["kickoff_time"].iloc[0]
    fixture_stats.loc[fixture_stats.index[-1], "was_home"] = this_gw["is_home"].iloc[0].astype(int)
    if this_gw["is_home"].iloc[0]: 
        fixture_stats.loc[fixture_stats.index[-1], "opponent_team"] = this_gw["team_a"].iloc[0]
    else: 
        fixture_stats.loc[fixture_stats.index[-1], "opponent_team"] = this_gw["team_h"].iloc[0]
    fixture_stats["kickoff_time"] = pd.to_datetime(fixture_stats["kickoff_time"])
    # filter out games that have not yet occurred
    # fixture_stats = fixture_stats[fixture_stats["kickoff_time"] < pd.to_datetime("12-25-2023").tz_localize("UTC")]
    # fixture_stats = fixture_stats[fixture_stats["kickoff_time"] < pd.Timestamp.now().tz_localize("UTC")]
    # get the most recent 3 games
    fixture_stats = fixture_stats.sort_values(by="kickoff_time").tail(4)
    fixture_stats.insert(0, column="name", value=[player_name] * fixture_stats.shape[0])
    to_add_df = pd.concat([to_add_df, fixture_stats], ignore_index=True)
to_add_df

,name,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,...,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out
0,Ederson Santana de Moraes,352,167,8,0,1,2023-12-16 15:00:00+00:00,2,2,17,...,1,0.00,0.00,0.00,1.82,55,-5808,1810790,37882,43690
1,Ederson Santana de Moraes,352,187,9,2,0,2023-12-27 20:15:00+00:00,1,3,19,...,1,0.00,0.00,0.00,1.01,55,-6127,1689592,17441,23568
2,Ederson Santana de Moraes,352,196,17,6,1,2023-12-30 15:00:00+00:00,2,0,20,...,1,0.00,0.00,0.00,0.33,55,18559,1715985,47372,28813
3,Ederson Santana de Moraes,0,0,15,0,0,2024-01-13 17:30:00+00:00,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Pedro Porro,506,178,9,2,1,2023-12-23 15:00:00+00:00,2,1,18,...,1,0.09,0.01,0.10,1.95,55,402771,2520492,418226,15455
5,Pedro Porro,506,184,5,5,0,2023-12-28 19:30:00+00:00,4,2,19,...,1,0.11,0.54,0.65,3.70,56,154549,2626444,179849,25300
6,Pedro Porro,506,198,3,2,1,2023-12-31 14:00:00+00:00,3,1,20,...,1,0.03,0.10,0.13,1.60,57,205930,2844708,237576,31646
7,Pedro Porro,0,0,14,0,0,2024-01-14 16:30:00+00:00,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,James Tarkowski,265,178,18,1,0,2023-12-23 15:00:00+00:00,2,1,18,...,1,0.00,0.00,0.00,1.34,45,65708,504265,78742,13034
9,James Tarkowski,265,187,13,1,1,2023-12-27 20:15:00+00:00,1,3,19,...,1,0.04,0.00,0.04,2.60,45,44516,546644,60156,15640


In [35]:
to_add_df.columns

Index(['name', 'element', 'fixture', 'opponent_team', 'total_points',
       'was_home', 'kickoff_time', 'team_h_score', 'team_a_score', 'round',
       'minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity',
       'threat', 'ict_index', 'starts', 'expected_goals', 'expected_assists',
       'expected_goal_involvements', 'expected_goals_conceded', 'value',
       'transfers_balance', 'selected', 'transfers_in', 'transfers_out'],
      dtype='object')

In [36]:
full_data = pd.read_csv("final_data_official.csv")
full_data

,name,club,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,...,ict_index,value,transfers_balance,selected,transfers_in,transfers_out,position,season,opponent_team_name,result
0,Aaron Cresswell,West Ham United,454,10,4,0,False,2016-08-15 19:00:00+00:00,2.0,1.0,...,0.0,55,0,14023,0,0,Defender,2016/2017,Chelsea,L
1,Aaron Lennon,Everton,142,3,17,1,True,2016-08-13 14:00:00+00:00,1.0,1.0,...,0.9,60,0,13918,0,0,Midfielder,2016/2017,Tottenham Hotspur,D
2,Abdoulaye Doucouré,Watford,482,7,13,0,False,2016-08-13 14:00:00+00:00,1.0,1.0,...,0.0,50,0,1051,0,0,Midfielder,2016/2017,Southampton,D
3,Adam Forshaw,Middlesbrough,286,6,14,1,True,2016-08-13 14:00:00+00:00,1.0,1.0,...,0.3,45,0,2723,0,0,Midfielder,2016/2017,Stoke City,D
4,Adam Lallana,Liverpool,205,8,1,11,False,2016-08-14 15:00:00+00:00,3.0,4.0,...,14.2,70,0,155525,0,0,Midfielder,2016/2017,Arsenal,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109075,Justin Hubner,Wolverhampton Wanderers,751,149,6,0,True,2023-12-05 19:30:00+00:00,1.0,0.0,...,0.0,40,344,555,375,31,Defender,2023/2024,Burnley,W
109076,Justin Hubner,Wolverhampton Wanderers,751,160,16,0,True,2023-12-09 15:00:00+00:00,1.0,1.0,...,0.0,40,329,1078,435,106,Defender,2023/2024,Nottingham Forest,D
109077,Justin Hubner,Wolverhampton Wanderers,751,170,19,0,False,2023-12-17 14:00:00+00:00,3.0,0.0,...,0.0,40,229,1583,418,189,Defender,2023/2024,West Ham United,L
109078,Justin Hubner,Wolverhampton Wanderers,751,180,7,0,True,2023-12-24 13:00:00+00:00,2.0,1.0,...,0.0,40,42,1763,197,155,Defender,2023/2024,Chelsea,W


In [37]:
shared_cols = list(to_add_df.columns.intersection(full_data.columns))
# shared_cols = shared_cols + ["team_h_score", "team_a_score"]
shared_cols

['name',
 'element',
 'fixture',
 'opponent_team',
 'total_points',
 'was_home',
 'kickoff_time',
 'team_h_score',
 'team_a_score',
 'round',
 'minutes',
 'goals_scored',
 'assists',
 'clean_sheets',
 'goals_conceded',
 'own_goals',
 'penalties_saved',
 'penalties_missed',
 'yellow_cards',
 'red_cards',
 'saves',
 'bonus',
 'bps',
 'influence',
 'creativity',
 'threat',
 'ict_index',
 'value',
 'transfers_balance',
 'selected',
 'transfers_in',
 'transfers_out']

In [38]:
# to_add_df = to_add_df.drop(columns=to_add_df.columns.difference(shared_cols))
# to_add_df

In [39]:
# converting columns to the correct data types
are_numeric = list(full_data.select_dtypes(include=['number']).columns)
to_add_df[are_numeric] = to_add_df[are_numeric].apply(pd.to_numeric)
to_add_df["was_home"] = to_add_df["was_home"].astype(bool)
to_add_df.dtypes

name                                       object
element                                     int64
fixture                                     int64
opponent_team                               int64
total_points                                int64
was_home                                     bool
kickoff_time                  datetime64[ns, UTC]
team_h_score                                int64
team_a_score                                int64
round                                       int64
minutes                                     int64
goals_scored                                int64
assists                                     int64
clean_sheets                                int64
goals_conceded                              int64
own_goals                                   int64
penalties_saved                             int64
penalties_missed                            int64
yellow_cards                                int64
red_cards                                   int64


In [40]:
# function that assigns the season to each row
def assign_season(ko_time):

    dates = ["2017-06-01", "2018-06-01", "2019-06-01", "2020-06-01", "2021-06-01", "2022-06-01", "2023-06-01", "2024-06-01"]

    for date in dates: 

        if ko_time < pd.to_datetime(date).tz_localize("UTC"): 

            year1 = int(date.split("-")[0])
            year0 = year1 - 1

            season = f"{year0}/{year1}"
            
            return season

In [41]:
to_add_df['season'] = to_add_df["kickoff_time"].apply(assign_season)

In [42]:
to_add_df.insert(1, "club", None)
to_add_df.insert(2, "position", None)
to_add_df

,name,club,position,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,...,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out,season
0,Ederson Santana de Moraes,None,None,352,167,8,0,True,2023-12-16 15:00:00+00:00,2,...,0.00,0.00,0.00,1.82,55,-5808,1810790,37882,43690,2023/2024
1,Ederson Santana de Moraes,None,None,352,187,9,2,False,2023-12-27 20:15:00+00:00,1,...,0.00,0.00,0.00,1.01,55,-6127,1689592,17441,23568,2023/2024
2,Ederson Santana de Moraes,None,None,352,196,17,6,True,2023-12-30 15:00:00+00:00,2,...,0.00,0.00,0.00,0.33,55,18559,1715985,47372,28813,2023/2024
3,Ederson Santana de Moraes,None,None,0,0,15,0,False,2024-01-13 17:30:00+00:00,0,...,0,0,0,0,0,0,0,0,0,2023/2024
4,Pedro Porro,None,None,506,178,9,2,True,2023-12-23 15:00:00+00:00,2,...,0.09,0.01,0.10,1.95,55,402771,2520492,418226,15455,2023/2024
5,Pedro Porro,None,None,506,184,5,5,False,2023-12-28 19:30:00+00:00,4,...,0.11,0.54,0.65,3.70,56,154549,2626444,179849,25300,2023/2024
6,Pedro Porro,None,None,506,198,3,2,True,2023-12-31 14:00:00+00:00,3,...,0.03,0.10,0.13,1.60,57,205930,2844708,237576,31646,2023/2024
7,Pedro Porro,None,None,0,0,14,0,False,2024-01-14 16:30:00+00:00,0,...,0,0,0,0,0,0,0,0,0,2023/2024
8,James Tarkowski,None,None,265,178,18,1,False,2023-12-23 15:00:00+00:00,2,...,0.00,0.00,0.00,1.34,45,65708,504265,78742,13034,2023/2024
9,James Tarkowski,None,None,265,187,13,1,True,2023-12-27 20:15:00+00:00,1,...,0.04,0.00,0.04,2.60,45,44516,546644,60156,15640,2023/2024


In [43]:
# all unique players
unique_players = np.unique(to_add_df["name"])
unique_elements = np.unique(to_add_df["element"])
current_szn = "2023/2024"
new_elements = []

for player_name in unique_players:

    # use whatever club and position is already in full_data to fill to_add_df
    try:
        ele_df = full_data[(full_data["name"] == player_name) & (full_data["season"] == current_szn)]
        position = np.unique(ele_df["position"])[0]
        club = np.unique(ele_df["club"])[0]
        
        to_add_df.loc[(to_add_df['name'] == player_name) & (to_add_df['season'] == current_szn), 'position'] = position
        to_add_df.loc[(to_add_df['name'] == player_name) & (to_add_df['season'] == current_szn), 'club'] = club

    except IndexError: 

        new_elements.append(ele)
        print(f"Player with ID {ele} not found. Moving on to next player.")
        continue

In [44]:
to_add_df

,name,club,position,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,...,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out,season
0,Ederson Santana de Moraes,Manchester City,Goalkeeper,352,167,8,0,True,2023-12-16 15:00:00+00:00,2,...,0.00,0.00,0.00,1.82,55,-5808,1810790,37882,43690,2023/2024
1,Ederson Santana de Moraes,Manchester City,Goalkeeper,352,187,9,2,False,2023-12-27 20:15:00+00:00,1,...,0.00,0.00,0.00,1.01,55,-6127,1689592,17441,23568,2023/2024
2,Ederson Santana de Moraes,Manchester City,Goalkeeper,352,196,17,6,True,2023-12-30 15:00:00+00:00,2,...,0.00,0.00,0.00,0.33,55,18559,1715985,47372,28813,2023/2024
3,Ederson Santana de Moraes,Manchester City,Goalkeeper,0,0,15,0,False,2024-01-13 17:30:00+00:00,0,...,0,0,0,0,0,0,0,0,0,2023/2024
4,Pedro Porro,Tottenham Hotspur,Defender,506,178,9,2,True,2023-12-23 15:00:00+00:00,2,...,0.09,0.01,0.10,1.95,55,402771,2520492,418226,15455,2023/2024
5,Pedro Porro,Tottenham Hotspur,Defender,506,184,5,5,False,2023-12-28 19:30:00+00:00,4,...,0.11,0.54,0.65,3.70,56,154549,2626444,179849,25300,2023/2024
6,Pedro Porro,Tottenham Hotspur,Defender,506,198,3,2,True,2023-12-31 14:00:00+00:00,3,...,0.03,0.10,0.13,1.60,57,205930,2844708,237576,31646,2023/2024
7,Pedro Porro,Tottenham Hotspur,Defender,0,0,14,0,False,2024-01-14 16:30:00+00:00,0,...,0,0,0,0,0,0,0,0,0,2023/2024
8,James Tarkowski,Everton,Defender,265,178,18,1,False,2023-12-23 15:00:00+00:00,2,...,0.00,0.00,0.00,1.34,45,65708,504265,78742,13034,2023/2024
9,James Tarkowski,Everton,Defender,265,187,13,1,True,2023-12-27 20:15:00+00:00,1,...,0.04,0.00,0.04,2.60,45,44516,546644,60156,15640,2023/2024


In [45]:
# CSV file that corresponds each club to an ID
opps = pd.read_csv("opps_final.csv")

In [46]:
# creating mapping from ID to team name
season_df = opps[opps["season"] == current_szn]
mapping = dict(zip(season_df["team"], season_df["opponent_team"]))

In [47]:
to_add_df["opponent_team_name"] = to_add_df["opponent_team"].map(mapping)

In [48]:
to_add_df["result"] = None
to_add_df

,name,club,position,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,...,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out,season,opponent_team_name,result
0,Ederson Santana de Moraes,Manchester City,Goalkeeper,352,167,8,0,True,2023-12-16 15:00:00+00:00,2,...,0.00,1.82,55,-5808,1810790,37882,43690,2023/2024,Crystal Palace,None
1,Ederson Santana de Moraes,Manchester City,Goalkeeper,352,187,9,2,False,2023-12-27 20:15:00+00:00,1,...,0.00,1.01,55,-6127,1689592,17441,23568,2023/2024,Everton,None
2,Ederson Santana de Moraes,Manchester City,Goalkeeper,352,196,17,6,True,2023-12-30 15:00:00+00:00,2,...,0.00,0.33,55,18559,1715985,47372,28813,2023/2024,Sheffield United,None
3,Ederson Santana de Moraes,Manchester City,Goalkeeper,0,0,15,0,False,2024-01-13 17:30:00+00:00,0,...,0,0,0,0,0,0,0,2023/2024,Newcastle United,None
4,Pedro Porro,Tottenham Hotspur,Defender,506,178,9,2,True,2023-12-23 15:00:00+00:00,2,...,0.10,1.95,55,402771,2520492,418226,15455,2023/2024,Everton,None
5,Pedro Porro,Tottenham Hotspur,Defender,506,184,5,5,False,2023-12-28 19:30:00+00:00,4,...,0.65,3.70,56,154549,2626444,179849,25300,2023/2024,Brighton & Hove Albion,None
6,Pedro Porro,Tottenham Hotspur,Defender,506,198,3,2,True,2023-12-31 14:00:00+00:00,3,...,0.13,1.60,57,205930,2844708,237576,31646,2023/2024,Bournemouth,None
7,Pedro Porro,Tottenham Hotspur,Defender,0,0,14,0,False,2024-01-14 16:30:00+00:00,0,...,0,0,0,0,0,0,0,2023/2024,Manchester United,None
8,James Tarkowski,Everton,Defender,265,178,18,1,False,2023-12-23 15:00:00+00:00,2,...,0.00,1.34,45,65708,504265,78742,13034,2023/2024,Tottenham Hotspur,None
9,James Tarkowski,Everton,Defender,265,187,13,1,True,2023-12-27 20:15:00+00:00,1,...,0.04,2.60,45,44516,546644,60156,15640,2023/2024,Manchester City,None


In [49]:
# function that propagates the result col of dataframe df with W, D, or L
def fill_result(df): 
    df.loc[(df["was_home"]) & (df["team_h_score"] > df["team_a_score"]), "result"] = "W"
    df.loc[(df["was_home"]) & (df["team_h_score"] < df["team_a_score"]), "result"] = "L"
    df.loc[(df["team_h_score"] == df["team_a_score"]), "result"] = "D"
    df.loc[~df["was_home"] & (df["team_h_score"] < df["team_a_score"]), "result"] = "W"
    df.loc[~df["was_home"] & (df["team_h_score"] > df["team_a_score"]), "result"] = "L"

In [50]:
fill_result(to_add_df)
to_add_df

,name,club,position,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,...,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out,season,opponent_team_name,result
0,Ederson Santana de Moraes,Manchester City,Goalkeeper,352,167,8,0,True,2023-12-16 15:00:00+00:00,2,...,0.00,1.82,55,-5808,1810790,37882,43690,2023/2024,Crystal Palace,D
1,Ederson Santana de Moraes,Manchester City,Goalkeeper,352,187,9,2,False,2023-12-27 20:15:00+00:00,1,...,0.00,1.01,55,-6127,1689592,17441,23568,2023/2024,Everton,W
2,Ederson Santana de Moraes,Manchester City,Goalkeeper,352,196,17,6,True,2023-12-30 15:00:00+00:00,2,...,0.00,0.33,55,18559,1715985,47372,28813,2023/2024,Sheffield United,W
3,Ederson Santana de Moraes,Manchester City,Goalkeeper,0,0,15,0,False,2024-01-13 17:30:00+00:00,0,...,0,0,0,0,0,0,0,2023/2024,Newcastle United,D
4,Pedro Porro,Tottenham Hotspur,Defender,506,178,9,2,True,2023-12-23 15:00:00+00:00,2,...,0.10,1.95,55,402771,2520492,418226,15455,2023/2024,Everton,W
5,Pedro Porro,Tottenham Hotspur,Defender,506,184,5,5,False,2023-12-28 19:30:00+00:00,4,...,0.65,3.70,56,154549,2626444,179849,25300,2023/2024,Brighton & Hove Albion,L
6,Pedro Porro,Tottenham Hotspur,Defender,506,198,3,2,True,2023-12-31 14:00:00+00:00,3,...,0.13,1.60,57,205930,2844708,237576,31646,2023/2024,Bournemouth,W
7,Pedro Porro,Tottenham Hotspur,Defender,0,0,14,0,False,2024-01-14 16:30:00+00:00,0,...,0,0,0,0,0,0,0,2023/2024,Manchester United,D
8,James Tarkowski,Everton,Defender,265,178,18,1,False,2023-12-23 15:00:00+00:00,2,...,0.00,1.34,45,65708,504265,78742,13034,2023/2024,Tottenham Hotspur,L
9,James Tarkowski,Everton,Defender,265,187,13,1,True,2023-12-27 20:15:00+00:00,1,...,0.04,2.60,45,44516,546644,60156,15640,2023/2024,Manchester City,L


In [25]:
# features / params I am unsure of - keeping just in case
# to_keep = ["name", "element", "opponent_team", "season", "kickoff_time"]
# target variable
# target = ["total_points"]
# specifically important features: position, club, ict_index, minutes, selected, result, transfers_balance, was_home, opponent_team_name
# features = ["position", "assists", "club", "clean_sheets", "goals_scored", "ict_index", "minutes", "red_cards", "saves", 
            #"selected", "result", "transfers_balance", "was_home", "opponent_team_name"]

In [26]:
# to_add_df = to_add_df[to_keep + target + features]
# to_add_df

,name,element,opponent_team,season,kickoff_time,total_points,position,assists,club,clean_sheets,goals_scored,ict_index,minutes,red_cards,saves,selected,result,transfers_balance,was_home,opponent_team_name
0,Ederson Santana de Moraes,352,8,2023/2024,2023-12-16 15:00:00+00:00,0,Goalkeeper,0,Manchester City,0,0,0.1,90,0,0,1810790,D,-5808,True,Crystal Palace
1,Ederson Santana de Moraes,352,9,2023/2024,2023-12-27 20:15:00+00:00,2,Goalkeeper,0,Manchester City,0,0,1.6,90,0,1,1689592,W,-6127,False,Everton
2,Ederson Santana de Moraes,352,17,2023/2024,2023-12-30 15:00:00+00:00,6,Goalkeeper,0,Manchester City,1,0,2.0,90,0,2,1715985,W,18559,True,Sheffield United
3,Ederson Santana de Moraes,0,15,2023/2024,2024-01-13 17:30:00+00:00,0,Goalkeeper,0,Manchester City,0,0,0.0,0,0,0,0,D,0,False,Newcastle United
4,Pedro Porro,506,9,2023/2024,2023-12-23 15:00:00+00:00,2,Defender,0,Tottenham Hotspur,0,0,4.3,90,0,0,2520492,W,402771,True,Everton
5,Pedro Porro,506,5,2023/2024,2023-12-28 19:30:00+00:00,5,Defender,1,Tottenham Hotspur,0,0,9.6,90,0,0,2626444,L,154549,False,Brighton & Hove Albion
6,Pedro Porro,506,3,2023/2024,2023-12-31 14:00:00+00:00,2,Defender,0,Tottenham Hotspur,0,0,6.0,90,0,0,2844708,W,205930,True,Bournemouth
7,Pedro Porro,0,14,2023/2024,2024-01-14 16:30:00+00:00,0,Defender,0,Tottenham Hotspur,0,0,0.0,0,0,0,0,D,0,False,Manchester United
8,James Tarkowski,265,18,2023/2024,2023-12-23 15:00:00+00:00,1,Defender,0,Everton,0,0,2.3,90,0,0,504265,L,65708,False,Tottenham Hotspur
9,James Tarkowski,265,13,2023/2024,2023-12-27 20:15:00+00:00,1,Defender,0,Everton,0,0,2.9,90,0,0,546644,L,44516,True,Manchester City


In [51]:
to_add_df.to_csv("gw_21_predictions_input.csv", index=False)